In [3]:
# -*- coding: utf-8 -*-
import dataiku
import pandas as pd, numpy as np
from dataiku import pandasutils as pdu

# Read recipe inputs
MATCHES_1_TO_N_FOR_MANUAL_REVIEW_STAGING = dataiku.Dataset("MATCHES_1_TO_N_FOR_MANUAL_REVIEW_STAGING")
MATCHES_1_TO_N_FOR_MANUAL_REVIEW_STAGING_df = MATCHES_1_TO_N_FOR_MANUAL_REVIEW_STAGING.get_dataframe()

MATCHES_1_TO_1_STAGING = dataiku.Dataset("MATCHES_1_TO_1_STAGING")
MATCHES_1_TO_1_STAGING_df = MATCHES_1_TO_1_STAGING.get_dataframe()

In [5]:
df_1_N = MATCHES_1_TO_N_FOR_MANUAL_REVIEW_STAGING_df
print(len(df_1_N))
df_1_N.head()

106


,CUSTOMER,MATCH_CUSTOMER,DRAW_UP_DATE
0,130199-BROWN & CALDWELL,"['BROWN AND CALDWELL', 'CALDWELL ELECTRICAL CO...","[Timestamp('2021-01-01 00:00:00+0000', tz='UTC..."
1,ZORN COMPRESSOR & EQUIPMENT IN,"['METROPOLITAN AIR COMPRESSOR CO INC', 'NATION...","[Timestamp('2019-02-01 00:00:00+0000', tz='UTC..."
2,WORLDWIDE WINE & SPIRITS INC,"['FISHER59 OKLAHOMA WINE SPIRITS LLP', 'WORLD...","[Timestamp('2019-11-01 00:00:00+0000', tz='UTC..."
3,WEST YAVAPAI GUIDANCE CLINI (2,['DELAWARE GUIDANCE SERVICES FOR CHILDREN AND ...,"[Timestamp('2019-04-01 00:00:00+0000', tz='UTC..."
4,WAVE FORM SYSTEMS INC,"['CONCRETE FORM ERECTORS INC', 'FORM WAVE SYST...","[Timestamp('2019-02-01 00:00:00+0000', tz='UTC..."


In [6]:
df_1_1 = MATCHES_1_TO_1_STAGING_df
print(len(df_1_1))
df_1_1.head()

1154


,CUSTOMER,MATCH_CUSTOMER,DRAW_UP_DATE
0,0972 HEINEKEN USA INC (PHH35),0972 HEINEKEN USA INC (PHH47),2019-06-01 00:00:00+00:00
1,ZOOM SEWER & DRAIN CLEANING SE,ZOOM SEWER AND DRAIN CLEANING SERVICE INC,2019-05-01 00:00:00+00:00
2,ZERNCO INC,ZERNCO,2020-07-01 00:00:00+00:00
3,ZASTROW THE BEER MAN,ZASTROW THE BEER MAN INC,2021-10-01 00:00:00+00:00
4,YOUR WAY FUMIGATION INC,FUMIGATION SERVICE SUPPLY INC,2022-02-01 00:00:00+00:00


In [19]:
from difflib import SequenceMatcher
import Levenshtein

idx = 0
exception_count = 0

_verified = []
_verified_match = []

_exception = []
_exception_match = []

for index, row in df_1_1.iterrows():

    idx+=1
    
    customer = row['CUSTOMER']
    match_customer = row['MATCH_CUSTOMER']
    
    r = Levenshtein.ratio(customer, match_customer)
    if r<0.8:
        _exception.append(customer)
        _exception_match.append(match_customer)
    else:
        _verified.append(customer)
        _verified_match.append(match_customer)

In [26]:
_CUSTOMER = []
_CUSTOMER_CLC = []

for i in range(0, len(_verified)):
    
    _CUSTOMER.append(_verified[i])
    _CUSTOMER.append(_verified_match[i])
    _CUSTOMER_CLC.append(_verified[i])
    _CUSTOMER_CLC.append(_verified[i])
    
df_clc = pd.DataFrame(_CUSTOMER, columns=['CUSTOMER'])
df_clc['CUSTOMER_CLC'] = _CUSTOMER_CLC
print(len(df_clc))
df_clc.tail(10)

1360


,CUSTOMER,CUSTOMER_CLC
1350,A FOLINO CONSTRUCTION INC,A FOLINO CONSTRUCTION INC
1351,ANTHONY D FOLINO CONSTRUCTION INC,A FOLINO CONSTRUCTION INC
1352,A C RIMMER INC,A C RIMMER INC
1353,AC RIMMER INC,A C RIMMER INC
1354,800 HEATERS INC,800 HEATERS INC
1355,1 800 HEATERS INC,800 HEATERS INC
1356,302102-JCDECAUX NA,302102-JCDECAUX NA
1357,302102 JCDECAYX NA,302102-JCDECAUX NA
1358,21ST CENTURY EQUIPMENT,21ST CENTURY EQUIPMENT
1359,21ST CENTURY EQUIPMENT LLC,21ST CENTURY EQUIPMENT


In [28]:
# Compute recipe outputs
# TODO: Write here your actual code that computes the outputs
# NB: DSS supports several kinds of APIs for reading and writing data. Please see doc.

MATCHES_VERIFIED_df = df_clc

# Write recipe outputs
MATCHES_VERIFIED = dataiku.Dataset("MATCHES_VERIFIED")
MATCHES_VERIFIED.write_with_schema(MATCHES_VERIFIED_df)

1360 rows successfully written (PK8OqO2KX8)
